In [36]:
import pandas as pd

train = pd.read_csv("swbd_disf_train_1_data.csv") #usecols = [i for i in range(n)]

validation = pd.read_csv("swbd_disf_heldout_data.csv")

test = pd.read_csv("swbd_disf_test_data.csv")

In [36]:
# need to use preprocessing from bert_crf file

In [41]:
sentences = [[word[0] for word in sentence] for sentence in train5]
sentences[0]

sent_t = [[word[0] for word in sentence] for sentence in test5]
sent_val = [[word[0] for word in sentence] for sentence in validation5]
labels = [[s[2] for s in sentence] for sentence in train5]

lab_t = [[s[2] for s in sentence] for sentence in test5]
lab_val = [[s[2] for s in sentence] for sentence in validation5]



In [42]:
#tag_values = list(set(data["Tag"].values))
tag_values = list(set([item for sublist in train5+validation5 for _,_, item in sublist]))
tag_values.append("PAD")
tag2idx = {t: i for i, t in enumerate(tag_values)}
id2tag= {i: t for i, t in enumerate(tag_values)}

In [45]:
! pip install transformers


     |████████████████████████████████| 2.9 MB 3.5 MB/s eta 0:00:01
     |████████████████████████████████| 288 kB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 56 kB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 109 kB 5.6 MB/s eta 0:00:01
     |████████████████████████████████| 140 kB 4.2 MB/s eta 0:00:01
  Attempting uninstall: ruamel.yaml
    Found existing installation: ruamel.yaml 0.15.100
    Uninstalling ruamel.yaml-0.15.100:
      Successfully uninstalled ruamel.yaml-0.15.100
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
deeppavlov 0.17.1 requires ruamel.yaml==0.15.100, but you have ruamel-yaml 0.17.16 which is incompatible.


In [47]:
! pip install torch

     |████████████████████████████████| 218.7 MB 33 kB/s  eta 0:00:015  |▎                               | 1.9 MB 2.9 MB/s eta 0:01:14     |█▏                              | 8.0 MB 4.0 MB/s eta 0:00:53     |███▉                            | 26.3 MB 929 kB/s eta 0:03:27     |█████████▍                      | 63.9 MB 1.7 MB/s eta 0:01:34     |██████████████▊                 | 100.5 MB 2.6 MB/s eta 0:00:46     |███████████████▉                | 108.5 MB 1.7 MB/s eta 0:01:07     |████████████████▏               | 110.8 MB 1.7 MB/s eta 0:01:05     |████████████████████▊           | 141.5 MB 2.3 MB/s eta 0:00:35     |█████████████████████▏          | 144.5 MB 681 kB/s eta 0:01:49     |█████████████████████▌          | 146.7 MB 3.8 MB/s eta 0:00:19     |█████████████████████▊          | 148.7 MB 2.1 MB/s eta 0:00:34     |██████████████████████▉         | 156.0 MB 3.2 MB/s eta 0:00:20     |████████████████████████        | 164.4 MB 2.5 MB/s eta 0:00:22     |██████████████████████████████▎ | 20

In [48]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

torch.__version__



Using TensorFlow backend.


'1.9.1'

In [49]:
MAX_LEN = 75
bs = 32

In [50]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()


In [ ]:
torch.cuda.get_device_name(0)


In [52]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)


Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [53]:
def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [54]:
tokenized_texts_and_labels = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(sentences, labels)
]

tokenized_texts_and_labels_test = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(sent_t, lab_t)
]

tokenized_texts_and_labels_val = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(sent_val, lab_val)
]

In [55]:
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

tokenized_texts_t = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels_test]
labels_t = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels_test]

tokenized_texts_val = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels_val]
labels_val = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels_val]


In [56]:
print(tokenized_texts[9829])

['well', 'i']


In [57]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

input_ids_t = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_t],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

In [58]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")


In [59]:
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]


In [60]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags,
                                                            random_state=2018, test_size=0.00001)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.00001)



In [61]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)


In [62]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)



In [63]:
import transformers
from transformers import BertForTokenClassification, AdamW

transformers.__version__


'4.11.2'

In [64]:
model = BertForTokenClassification.from_pretrained(
    "bert-base-cased",
    num_labels=len(tag2idx),
    output_attentions = False,
    output_hidden_states = False
)


Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [ ]:
model.cuda();


In [66]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=3e-5,
    eps=1e-8
)


In [67]:
from transformers import get_linear_schedule_with_warmup

epochs = 1
max_grad_norm = 1.0

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

In [68]:
! pip install seqeval


     |████████████████████████████████| 43 kB 928 kB/s eta 0:00:01
     |████████████████████████████████| 7.9 MB 5.1 MB/s eta 0:00:01
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16170 sha256=eea84cf54cb4f2ff6a8c6fca5a4ccfb737a8c3d0cebb871c1917e629a8f53b9f
  Stored in directory: /Users/mortro/Library/Caches/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.21.2
    Uninstalling scikit-learn-0.21.2:
      Successfully uninstalled scikit-learn-0.21.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dtreeviz 1.3 requires graphviz>=0.9, but you have graphviz 0.8.4 which is incompatible.
deeppavlov 0.17.1 requires ruamel.yaml==0.15.100, but you have ruamel-yaml 0.17.16 which is incompatible.
deeppavlov 0.17

In [69]:
from seqeval.metrics import f1_score, accuracy_score
from tqdm import tqdm, trange


In [70]:
from tqdm import tqdm
## Store the average loss after each epoch so we can plot them.
loss_values, validation_loss_values = [], []

for _ in trange(epochs, desc="Epoch"):
    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.

    # Put the model into training mode.
    model.train()
    # Reset the total loss for this epoch.
    total_loss = 0
    loop = tqdm(enumerate(train_dataloader), total=len(train_dataloader), leave=False)
    # Training loop
    for step, batch in loop:
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # Always clear any previously calculated gradients before performing a backward pass.
        model.zero_grad()
        # forward pass
        # This will return the loss (rather than the model output)
        # because we have provided the `labels`.
        outputs = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask, labels=b_labels)
        # get the loss
        loss = outputs[0]
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # track train loss
        total_loss += loss.item()
        # Clip the norm of the gradient
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))

    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)


    print()

Epoch: 100%|██████████| 1/1 [16:36:25<00:00, 59785.56s/it]

Average train loss: 0.2133776111603363



In [76]:
torch.save(model,"\.")